In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [86]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [87]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
672,(2023/02/28) 23-24,50.5,40
673,Min,50.5,0
674,Max,239.9,40
675,Average,95.47619047619047619047619048,19.285714285714285714285714286


In [88]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/03/06) 04-05,29.00,50.0
6,(2017/03/06) 05-06,29.00,50.0
7,(2017/03/06) 06-07,65.00,0.0
8,(2017/03/06) 07-08,97.21,0.0
9,(2017/03/06) 08-09,97.21,0.0
...,...,...,...
668,(2023/02/28) 19-20,239.90,0.0
669,(2023/02/28) 20-21,96.00,0.0
670,(2023/02/28) 21-22,96.00,0.0
671,(2023/02/28) 22-23,50.50,40.0


In [89]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/03/06) 04-05,29.00,50.0,2017-03-06,4
6,(2017/03/06) 05-06,29.00,50.0,2017-03-06,5
7,(2017/03/06) 06-07,65.00,0.0,2017-03-06,6
8,(2017/03/06) 07-08,97.21,0.0,2017-03-06,7
9,(2017/03/06) 08-09,97.21,0.0,2017-03-06,8
...,...,...,...,...,...
668,(2023/02/28) 19-20,239.90,0.0,2023-02-28,19
669,(2023/02/28) 20-21,96.00,0.0,2023-02-28,20
670,(2023/02/28) 21-22,96.00,0.0,2023-02-28,21
671,(2023/02/28) 22-23,50.50,40.0,2023-02-28,22


In [90]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,29.00,50.0,2017-03-06,4
6,29.00,50.0,2017-03-06,5
7,65.00,0.0,2017-03-06,6
8,97.21,0.0,2017-03-06,7
9,97.21,0.0,2017-03-06,8
...,...,...,...,...
668,239.90,0.0,2023-02-28,19
669,96.00,0.0,2023-02-28,20
670,96.00,0.0,2023-02-28,21
671,50.50,40.0,2023-02-28,22


In [91]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0
2017-03-06,29.00,50.0,5,2017,3,6,0
2017-03-06,65.00,0.0,6,2017,3,6,0
2017-03-06,97.21,0.0,7,2017,3,6,0
2017-03-06,97.21,0.0,8,2017,3,6,0
...,...,...,...,...,...,...,...
2023-02-28,239.90,0.0,19,2023,2,28,1
2023-02-28,96.00,0.0,20,2023,2,28,1
2023-02-28,96.00,0.0,21,2023,2,28,1


In [92]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2023 2     24    320.0  16160.0             50.5     239.90       50.5   
           25    520.0  26260.0             50.5      96.00       50.5   
           26    960.0  48480.0             50.5      50.50       50.5   
           27    160.0   8080.0             50.5     239.90       50.5   
           28    320.0  16160.0             50.5     239.90       50.5   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2017 3     6     65.852500       50.0        0.0   10.000000 2017-03-06  
           7     59.710417       50.0        0.0   16.666667 2017-03-07  
           8     59.710417       50.0        0.0   16.666667 2017-03-08  
           9     59.710417       50.0        0.0   16.666667 2017-03-09  
           10    65.852500       50.0        0.0   10.000000 2017-03-10  
...                    ...        ...        ...         ...        ...  
2023 2     24   110.812500       40.0        0.0   13.333333 2023-02-24  
           25    66.425000       40.0        0.0   26.000000 2023-02-25  
           26    50.500000       40.0       40.0   40.000000 2023-02-26  
           27   122.875000       40.0        0.0    8.000000 2023-02-27  
           28   110.812500       40.0        0.0   13.333333 2023-02-28  

[2125 rows x 10 columns]

In [93]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0,1450.0,off
2017-03-06,29.00,50.0,5,2017,3,6,0,1450.0,standard
2017-03-06,65.00,0.0,6,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,7,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,8,2017,3,6,0,0.0,morning
...,...,...,...,...,...,...,...,...,...
2023-02-28,239.90,0.0,19,2023,2,28,1,0.0,evening
2023-02-28,96.00,0.0,20,2023,2,28,1,0.0,off
2023-02-28,96.00,0.0,21,2023,2,28,1,0.0,off


In [94]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2023 2     24    320.0  16160.0             50.5     239.90       50.5   
           25    520.0  26260.0             50.5      96.00       50.5   
           26    960.0  48480.0             50.5      50.50       50.5   
           27    160.0   8080.0             50.5     239.90       50.5   
           28    320.0  16160.0             50.5     239.90       50.5   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2017 3     6     65.852500       50.0        0.0   10.000000 2017-03-06  ...   
           7     59.710417       50.0        0.0   16.666667 2017-03-07  ...   
           8     59.710417       50.0        0.0   16.666667 2017-03-08  ...   
           9     59.710417       50.0        0.0   16.666667 2017-03-09  ...   
           10    65.852500       50.0        0.0   10.000000 2017-03-10  ...   
...                    ...        ...        ...         ...        ...  ...   
2023 2     24   110.812500       40.0        0.0   13.333333 2023-02-24  ...   
           25    66.425000       40.0        0.0   26.000000 2023-02-25  ...   
           26    50.500000       40.0       40.0   40.000000 2023-02-26  ...   
           27   122.875000       40.0        0.0    8.000000 2023-02-27  ...   
           28   110.812500       40.0        0.0   13.333333 2023-02-28  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 3     6                   3.0                 2.0       150.0   
           7                   3.0                 2.0       350.0   
           8                   3.0                 2.0       350.0   
           9                   3.0                 2.0       350.0   
           10                  3.0                 2.0       150.0   
...                            ...                 ...         ...   
2023 2     24                  3.0                 2.0       280.0   
           25                  NaN                 NaN       520.0   
           26                  NaN                 NaN       960.0   
           27                  3.0                 2.0       120.0   
           28                  3.0                 2.0       280.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 3     6               50.0             0.0             0.0     4350.0   
           7               50.0             0.0             0.0    10150.0   
           8               50.0             0.0             0.0    10150.0   
           9               50.0             0.0             0.0    10150.0   
           10              50.0             0.0             0.0     4350.0   
...                         ...             ...             ...        ...   
2023 2     24              40.0             0.0             0.0    14140.0   
           25               0.0             NaN             NaN    26260.0   
           26               NaN             NaN             NaN    48480.0   
           27              40.0             0.0             0.0     6060.0   
           28              40.0             0.0             0.0    14140.0   

                standard_value  morning_value  evening_value  
year month day         

In [95]:
daily.to_csv(out + pref + '_daily.csv')

In [96]:
hourly.to_csv(out + pref + '_hourly.csv')

In [97]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-03-06,4,29.00,50.0,off
2017-03-06,5,29.00,50.0,standard
2017-03-06,6,65.00,0.0,morning
2017-03-06,7,97.21,0.0,morning
2017-03-06,8,97.21,0.0,morning
...,...,...,...,...
2023-02-28,19,239.90,0.0,evening
2023-02-28,20,96.00,0.0,off
2023-02-28,21,96.00,0.0,off


In [98]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [99]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,470.884615,23061.866154,71.232500,29.95,58.295442,28.461538,10.423077,20.294551,...,40.0,48.975620,34.090350,61.625498,86.514416,115.321589,24.197880,11.700000,19.868778,7.550000
4,2017,7,460.096774,20451.535323,70.355000,25.05,52.786952,22.838710,11.580645,20.188172,...,42.0,44.450508,32.073682,59.895533,79.061769,110.973696,24.547486,10.317460,18.755102,5.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022,10,366.067742,16840.175419,124.000000,29.07,95.447099,33.570968,5.809677,15.938306,...,42.0,46.002894,45.505897,52.060186,NaN,NaN,29.294972,0.000000,3.512245,0.000000
68,2022,11,148.666667,13605.586667,206.282917,56.00,126.244069,13.333333,2.000000,6.400000,...,44.0,91.517399,91.552536,90.992857,NaN,NaN,12.666667,0.000000,1.129032,0.000000
69,2022,12,321.666667,15843.566667,130.496000,39.98,100.530563,21.666667,5.000000,13.875000,...,34.0,49.254611,46.964127,54.290345,51.408571,39.980000,18.194946,5.490196,11.717172,2.352941
70,2023,1,443.870968,20881.600000,122.875000,39.98,93.741054,40.000000,7.741935,19.397849,...,44.0,47.044302,47.289351,45.749032,39.980000,39.980000,35.200000,1.818182,5.000000,1.818182


In [100]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,470.884615,23061.866154,71.232500,29.95,58.295442,28.461538,10.423077,20.294551,...,40.0,48.975620,34.090350,61.625498,86.514416,115.321589,24.197880,11.700000,19.868778,7.550000
4,2017,7,460.096774,20451.535323,70.355000,25.05,52.786952,22.838710,11.580645,20.188172,...,42.0,44.450508,32.073682,59.895533,79.061769,110.973696,24.547486,10.317460,18.755102,5.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022,10,366.067742,16840.175419,124.000000,29.07,95.447099,33.570968,5.809677,15.938306,...,42.0,46.002894,45.505897,52.060186,NaN,NaN,29.294972,0.000000,3.512245,0.000000
68,2022,11,148.666667,13605.586667,206.282917,56.00,126.244069,13.333333,2.000000,6.400000,...,44.0,91.517399,91.552536,90.992857,NaN,NaN,12.666667,0.000000,1.129032,0.000000
69,2022,12,321.666667,15843.566667,130.496000,39.98,100.530563,21.666667,5.000000,13.875000,...,34.0,49.254611,46.964127,54.290345,51.408571,39.980000,18.194946,5.490196,11.717172,2.352941
70,2023,1,443.870968,20881.600000,122.875000,39.98,93.741054,40.000000,7.741935,19.397849,...,44.0,47.044302,47.289351,45.749032,39.980000,39.980000,35.200000,1.818182,5.000000,1.818182


In [101]:
monthly=monthly.set_index(['year','month'])

In [102]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

#monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
#monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
#monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
#monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [103]:
monthly.to_csv(out + pref + '_monthly.csv')

In [104]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [105]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,14303.691317,47.760260,123.375591,38.950860,71.580992,23.260215,4.966129,13.125408,46.532406,...,77.000000,443414.430833,345.166667,252.000000,66.5,44.333333,19.282879,8.811967,3.898153,1.694128
1,2,11539.427574,55.422794,136.552160,42.780556,82.328685,18.564815,2.722222,9.351651,52.910081,...,0.000000,311564.544500,309.833333,219.666667,57.5,38.333333,15.134484,4.587412,1.146759,0.000000
2,3,9296.853632,45.035008,124.015517,40.972299,73.526631,18.200000,3.419540,9.659646,48.832700,...,33.666667,269608.755333,321.666667,238.000000,63.0,42.000000,18.274391,4.832581,2.020063,1.122222
3,4,12947.154740,41.150806,106.156818,36.257403,62.987942,21.198701,5.056494,12.798074,42.367902,...,188.233333,332310.305000,288.000000,209.000000,55.0,36.666667,22.902126,13.194990,12.443687,12.139141
4,5,17561.798528,39.925687,98.518068,31.217670,57.067281,29.732955,8.563636,18.209574,37.186322,...,501.800000,515146.090167,330.333333,237.500000,62.5,41.666667,24.526472,14.758634,14.172815,14.617090
5,6,18130.223932,42.341918,128.855625,33.979261,70.181525,35.486364,6.700568,19.258305,42.162741,...,52.500000,531819.902000,327.333333,240.833333,63.5,42.333333,30.293179,10.768626,3.878331,1.309921
6,7,16401.192452,44.696218,144.999839,37.166613,79.035000,29.140323,5.445161,15.312979,47.026926,...,142.833333,508436.966000,346.500000,251.500000,66.0,44.000000,25.969333,6.692935,3.810156,3.287698
7,8,19070.078355,52.275256,151.115108,36.657097,78.016759,27.290323,6.402151,16.099960,45.399627,...,310.833333,591172.429000,342.666667,253.666667,67.0,44.666667,22.753137,10.839196,7.772508,6.757246
8,9,23163.265550,49.008260,124.431167,32.590444,66.993342,37.264444,7.773333,21.136037,40.359906,...,319.166667,694897.966500,337.666667,243.666667,64.0,42.666667,30.691547,13.456302,9.282744,7.901389
9,10,16266.489285,47.658616,109.032742,35.598925,66.793889,21.795699,7.056989,14.738607,43.746667,...,178.333333,504261.167833,346.500000,251.500000,66.0,44.000000,19.852348,11.775993,6.521410,4.021739


In [106]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [107]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                    
1     0      39.146667  20.911364   872.348265
      1      39.146667  20.911364   872.348265
      2      39.146667  20.911364   872.348265
      3      39.146667  20.911364   872.348265
      4      38.950860  20.847849   866.210317
...                ...        ...          ...
12    19    134.748883  11.788827   661.823631
      20     72.821285  20.551397  1124.578391
      21     72.821285  20.551397  1124.578391
      22     41.726872  39.601676  1652.008235
      23     41.726872  39.601676  1652.008235

[288 rows x 3 columns]

In [108]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [109]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

price     volume        value
year month hour                                    
2017 3     0      27.422941  50.000000  1323.629412
           1      27.422941  50.000000  1323.629412
           2      27.422941  50.000000  1323.629412
           3      27.422941  50.000000  1323.629412
           4      27.588947  50.000000  1336.931579
...                     ...        ...          ...
2023 2     19    192.285714   5.714286   288.571429
           20     83.000000  11.428571   577.142857
           21     83.000000  11.428571   577.142857
           22     50.500000  40.000000  2020.000000
           23     50.500000  40.000000  2020.000000

[1728 rows x 3 columns]

In [110]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [111]:
from statsmodels.tsa.stattools import pacf

In [112]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.739415,0.771181,0.773532,0.755956
2,-0.058182,-0.182031,-0.168011,-0.221636
3,0.246070,0.297018,0.298162,0.247327
4,0.161941,0.155371,0.166359,-0.046404
5,0.061229,0.145729,0.135051,0.098578
6,0.103881,0.035872,0.091507,0.107601
7,0.062471,-0.148793,-0.147376,0.206313
8,-0.009831,0.039964,0.041934,0.047099
9,-0.235606,-0.203438,-0.232074,-0.299846


In [113]:
pacfs.to_csv(out + pref + '_pacfs.csv')